<a href="https://colab.research.google.com/github/0ShNa0/Entailment_DATA/blob/main/vimpnewest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text1="U.S. president Obama would not deploy U.S. troops in Ukraine"


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp=spacy.load("en_core_web_lg")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
def get_entities(text):
  return[X.text for X in nlp(text).ents]
ent_list=get_entities(text1)
ent_set=set(ent_list)
entities=[entity for entity in ent_set]
print(entities)

['Obama', 'Ukraine', 'U.S.']


In [ ]:
def mask_occurrences(sentence, target_word):

    indices = [i for i in range(len(sentence)) if sentence.startswith(target_word, i)]

    if not indices:
        return [sentence]

    masked_sentences = [sentence[:i] + "<mask>" + sentence[i + len(target_word):] for i in indices]

    return masked_sentences


In [ ]:
def generate_input(ents, text):
    sent = []
    for ent in ents:
        res = mask_occurrences(text, ent)
        for r in res:
            sent.append(r)
    return sent
sent=generate_input(entities,text1)

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model="studio-ousia/luke-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
gen_list = []
punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
for s in sent:
    results = unmasker(s,top_k=5)
    gen_list.append([result['sequence'] for result in results if result['token_str'] not in punc])

def process_gen_list(gen_list):
    fin_list = []
    for i, results in enumerate(gen_list, start=1):
        for sequence in results:
            fin_list.append(sequence)
    return fin_list

result_list = process_gen_list(gen_list)


In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.2,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [ ]:
def postphrase(given_list):
    final = []
    for sentence in given_list:
        final.append(paraphrase(sentence))
    return final

ans = postphrase(result_list)


[['The president of the United States declares that they will not place U.S. troops in Ukraine.'], ['The deployment of U.S. troops in Ukraine is not a choice for Obama, as stated by the president.'], ['U.S. troops are not being deployed in Ukraine by President Trump, according to reports.'], ['U.S. troops are not being deployed in Ukraine by President Putin.'], ['Pence stated that U.S. troops would not be sent to Ukraine by the United States during the crisis.'], ['Obama declined to send U.S. troops in Afghanistan, unlike his predecessors.'], ['The deployment of U.S. troops in Iraq is not supported by President Obama.'], ['U.S. troops are not being sent to Syria by President Obama.'], ['Libyan troops would not be sent by U.S. president Obama, as stated in his statement.'], ['The deployment of U.S. troops in Kosovo is not supported by President Obama.'], ['The deployment of U.S. troops in Ukraine was not authorized by former president Obama.'], ['The deployment of U.S. troops in Ukraine